In [1]:
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense


import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array, load_img

dataset_path = "/kaggle/input/cleartest-final/The IQ-OTHNCCD lung cancer dataset/"

class_names = ["Infected Cases", "Normal cases"]
num_classes = len(class_names)

images = []
labels = []

for i, class_name in enumerate(class_names):
    class_path = os.path.join(dataset_path, class_name)
    for img_filename in os.listdir(class_path):
        img_path = os.path.join(class_path, img_filename)
        img = load_img(img_path, target_size=(100, 100)) 
        img_array = img_to_array(img)
        img_array = img_array / 255.0  
        images.append(img_array)
        labels.append(i)  

images = np.array(images)
labels = np.array(labels)

x_train, x_temp, y_train, y_temp = train_test_split(images, labels, test_size=0.2, random_state=42)
x_valid, x_test, y_valid, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=42)

print(f"Number of classes: {num_classes}")
print(f"Number of training samples: {len(x_train)}")
print(f"Number of validation samples: {len(x_valid)}")
print(f"Number of test samples: {len(x_test)}")


base_model = VGG16(weights='imagenet', include_top=False, input_shape=(100, 100, 3))

for layer in base_model.layers:
    layer.trainable = False

model = Sequential([
    base_model, 
    Flatten(),
    Dense(512, activation='relu'),
    Dense(256, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=Adam(learning_rate=0.001), loss=tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

history = model.fit(x_train, y_train, batch_size=32, epochs=10, validation_data=(x_valid, y_valid))

# Evaluate on test setq
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test Loss: {test_loss}, Test Accuracy: {test_acc}')

model.summary()


Number of classes: 2
Number of training samples: 29063
Number of validation samples: 3633
Number of test samples: 3633
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


KeyboardInterrupt: 